In [1]:
import pandas as pd
from datetime import datetime

In [5]:
def preprocess_data(file_path):

    data = pd.read_csv(file_path)
    data['time'] = pd.to_datetime(data['time'])

    press_events = data[data['event'] == 'press']
    
    #Calculating typing duration in minutes
    total_duration = (press_events['time'].max() - press_events['time'].min()).total_seconds() / 60.0
    #Calculating number of backspaces
    backspace_count = press_events[press_events['key'] == 'Key.backspace'].shape[0]
    #Calculating typing speed(per minute)
    typing_speed = press_events.shape[0] / total_duration
    #Calculating backspaces (per minute)
    backspaces_per_minute = backspace_count / total_duration
    
    return typing_speed, backspaces_per_minute


In [ ]:
typing_speed_owner, backspaces_per_minute_owner = preprocess_data('D:\\user_authentication\\test_m.csv')
typing_speed_non_owner, backspaces_per_minute_non_owner = preprocess_data('D:\\user_authentication\\test_a.csv')

data = {
    'typing_speed': [typing_speed_owner, typing_speed_non_owner],
    'backspaces_per_minute': [backspaces_per_minute_owner, backspaces_per_minute_non_owner],
    'label': [1, 0]  # 1 for owner, 0 for non-owner
}

df = pd.DataFrame(data)

## Classification model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df[['typing_speed', 'backspaces_per_minute']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Training a Neural Network Classifier
model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))
